In [1]:
# from structure import *
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.python.data import Dataset
from tensorflow.python import feature_column as fc
from tensorflow.python.keras.estimator import model_to_estimator
from tensorflow.python import estimator as est
from tensorflow.python.keras import Model, Input, Sequential
from tensorflow.python.keras.callbacks import *
from tensorflow.python.keras.layers import Dense

C:\Users\tales\Miniconda3\envs\Deep_Learning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
link = 'postgres://udmar9f385gb3o:p5c35862bd63c7c5070e64040171be67b0c6da0e18200d241c04e215329fd8cb3@ec2-18-234-15-254.compute-1.amazonaws.com:5432/dfcor1iq1tm3i7'
disk_engine = create_engine(link, echo=False)

sample = pd.read_sql('BTC_Train', con=disk_engine)

span = 120
tar_periods = 7

target_cat_cols = [f'{i+1}_per_tar_cat' for i in range(tar_periods)]

target_val_cols = [f'{i+1}_per_tar_val' for i in range(tar_periods)]

target_cols = target_cat_cols+target_val_cols

drop_cols = [
 'index',
 'id'
]

cat_cols = [
 'name',
 'symbol',
 'rank',
 'max_sell_ex',
 'min_sell_ex',
 'max_buy_ex',
 'min_buy_ex',
 'Event_Count',
 'Time',
 'Time_Year',
 'Time_Month',
 'Time_Week',
 'Time_Day',
 'Time_Dayofweek',
 'Time_Dayofyear',
 'Time_Hour',
 'Time_Minute',
 'Time_Is_month_end',
 'Time_Is_month_start',
 'Time_Is_quarter_end',
 'Time_Is_quarter_start',
 'Time_Is_year_end',
 'Time_Is_year_start',
 'Time_Elapsed'
]

con_cols = [col for col in list(sample) if col not in target_cols+drop_cols+cat_cols]


In [5]:
sample.shape

(967, 210)

In [21]:
full_test = pd.read_sql('Augmented_Train', con=disk_engine)

In [20]:
iter(full_test)

<generator object SQLTable._query_iterator at 0x00000293803C3570>

In [17]:
dataset = Dataset.from_generator(full_test, output_types=(tf.float32, tf.float32))

TypeError: `generator` must be callable.

In [ ]:
def data_processing(data, cat_cols, con_cols, tar_cols):
    return data[cat_cols+con_cols].values, data[tar_cols].values

def processing_wrapper(data, cat_cols, con_cols, tar_cols):
    return tf.py_func(func = data_processing,
                      inp = (data, cat_cols, con_cols, tar_cols),
                      Tout= (tf.float32, tf.float32))

In [ ]:
dataset = Dataset.from_generator(full_test,
                                 output_types=(tf.float32, tf.float32))

In [8]:
import pickle
def picklize(location, object):
    with open(location, 'wb') as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f'Pickle saved to {location}.')

def unpickle(location):
    with open(location, 'rb') as handle:
        return pickle.load(handle)

In [9]:
cat_mappings = unpickle("C:\\Users\\tales\Code\structured_analysis\cat_mapper.pkl")
con_mappings = unpickle("C:\\Users\\tales\Code\structured_analysis\con_mapper.pkl")

In [19]:
strk = Structure(full_test, target_val_cols, cat_cols, con_cols, tar_type='value')

In [21]:
strk.processing()

C:\Users\tales\Miniconda3\envs\Deep_Learning\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\tales\Miniconda3\envs\Deep_Learning\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [21]:
strk.embedding_train(2, 32, 0.001, .1, layers=[64, 128, 256], dropouts=[0.5, 0.25, 0.1], model='new',
                        shuffle=True, save=False, con_dim=10)

KeyboardInterrupt: 

In [ ]:
preds = strk.emb_model.predict(strk.emb_process(strk.data[cat_cols+con_cols]))

C:\Users\tales\Miniconda3\envs\Deep_Learning\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\tales\Miniconda3\envs\Deep_Learning\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
preds.shape

(7395, 1)